In [2]:
import numpy as np # linear algebra
import pandas as pd #
import glob
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import csv
import os


In [3]:

img_size = (224, 224)
img_array_list = []
cls_list = []

train_dir = 'input/1056lab-defect-detection/train/class'
train_def_dir ='input/DataAugmentation/Class'
for i in range(6):
    x = str(i + 1)
    img_list1 = glob.glob(train_dir + x + '/*.png')
    for i in img_list1:
        img = load_img(i, color_mode='grayscale', target_size=(img_size))
        img_array = img_to_array(img) / 255
        img_array_list.append(img_array)
        cls_list.append(0)

    img_list1 = glob.glob(train_def_dir + x + '_def/*.png')
    for i in img_list1:
        img = load_img(i, color_mode='grayscale', target_size=(img_size))
        img_array = img_to_array(img) / 255
        img_array_list.append(img_array)
        cls_list.append(1)

X_train = np.array(img_array_list)
y_train = np.array(cls_list)

In [7]:
X_train.shape

(12300, 224, 224, 1)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
import tensorflow.keras.optimizers as opt

model = Sequential()
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 1)))
model.add(MaxPooling2D(pool_size=(8, 8)))
model.add(Dropout(rate=0.5))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(rate=0.5))
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))

model.add(Dense(units=1, activation='sigmoid'))
Nadam = opt.Nadam(lr=8e-4, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
model.compile(optimizer=Nadam, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 128)     1280      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 27, 27, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [9]:
from keras.utils.np_utils import to_categorical
model.fit(X_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
193/193 [==============================] - 1437s 7s/step - loss: 0.6920 - accuracy: 0.5588
Epoch 2/10
193/193 [==============================] - 2048s 11s/step - loss: 0.5695 - accuracy: 0.6793
Epoch 3/10
193/193 [==============================] - 1392s 7s/step - loss: 0.4725 - accuracy: 0.7333
Epoch 4/10
193/193 [==============================] - 1037s 5s/step - loss: 0.4483 - accuracy: 0.7476
Epoch 5/10
193/193 [==============================] - 1043s 5s/step - loss: 0.4438 - accuracy: 0.7505
Epoch 6/10
193/193 [==============================] - 1031s 5s/step - loss: 0.4294 - accuracy: 0.7674
Epoch 7/10
193/193 [==============================] - 1001s 5s/step - loss: 0.4147 - accuracy: 0.7902
Epoch 8/10
193/193 [==============================] - 1074s 6s/step - loss: 0.4022 - accuracy: 0.8043
Epoch 9/10
193/193 [==============================] - 1328s 7s/step - loss: 0.3933 - accuracy: 0.8084
Epoch 10/10
193/193 [==============================] - 1082s 6s/step - loss: 0.37

In [12]:
import glob
from keras.preprocessing.image import load_img, img_to_array

img_array_list = []

img_list = glob.glob('input/1056lab-defect-detection/test/*.png')
img_list.sort()
for i in img_list:
    img = load_img(i, color_mode='grayscale', target_size=(img_size))
    img_array = img_to_array(img) / 255
    img_array_list.append(img_array)

X_test = np.array(img_array_list)

In [40]:
filenames =[]
with open('submission1.csv', 'w', newline='') as writeFile:
    writer = csv.writer(writeFile)
    filenames.append('filename')
    writer.writerow(filenames)
    for filename in os.listdir("input/1056lab-defect-detection/test"):
        filenames=[]
        filenames.append(filename)
        writer.writerow(filenames) 
writeFile.close()

In [41]:
predict = model.predict(X_test)[:, 0]
submit = pd.read_csv('submission1.csv')
submit['defect'] = predict
submit.to_csv('submission1.csv', index=False)